**Goal 1: ** Import a dataset and train classifiers ✅

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install fairlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)

{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

In [ ]:
#preprocess the dataset

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Define features and target
X = adult.data.features
y = (adult.data.targets["income"] == ">50K").astype(int)

# Identify sensitive features and other categorical/numerical features
sensitive_features = ["sex"]  # Example sensitive attributes
categorical_features = ["workclass", "marital-status", "occupation", "relationship", "native-country", "race"]
numerical_features = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]

X[X == '?'] = np.nan
X.info()

# Preprocessing:
# 1. Handle Missing Values (Imputation or removal)
X = X.dropna()
y = y.loc[X.index]

# Changing everything to integer models. Encode catagorical features, drop education (redundant)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for col in categorical_features:
    label_encoder.fit(X[col])
    X[col] = label_encoder.transform(X[col])
for col in sensitive_features:
    label_encoder.fit(X[col])
    X[col] = label_encoder.transform(X[col])
X = X.drop(columns=['education'])


<ipython-input-3-4225d2e571cd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X == '?'] = np.nan
<ipython-input-3-4225d2e571cd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X == '?'] = np.nan


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      46033 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  47985 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


<ipython-input-3-4225d2e571cd>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label_encoder.transform(X[col])
<ipython-input-3-4225d2e571cd>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label_encoder.transform(X[col])
<ipython-input-3-4225d2e571cd>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [ ]:
#test train split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# One-hot encode categorical features
#X_encoded = pd.get_dummies(X, drop_first=True)
sensitive_attr=X['sex']

X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(
    X, y, sensitive_attr, test_size=0.3, random_state=42, stratify=y
)

#Normalize the data
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

# Train Logistic Regression
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print(accuracy_score(y_test, y_pred_lr))

# Train Random Forest
rf = RandomForestClassifier(random_state=42, n_estimators=30)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(accuracy_score(y_test, y_pred_rf))

0.841158693889585
0.8473501879560699


**Goal 2:** The next goal is to compare and understand both these classifiers for fairness with respect to all three notions of fairness with sex as a protected charecterestic- https://fairlearn.org/v0.5.0/api_reference/fairlearn.metrics.html

1) Demographic Parity difference
2) Predictve rate parity difference/ equal opportunity difference: compares True Positive rates between different groups
3) Equalized odds difference ✅

In [ ]:
from fairlearn.metrics import MetricFrame, demographic_parity_difference, selection_rate, equalized_odds_difference
from sklearn.metrics import precision_score, recall_score

# Function to compute group fairness metrics
def compute_fairness_metrics(y_true, y_pred, sensitive_features):
    mf = MetricFrame(
        metrics={
            "Selection Rate": selection_rate,
            "Precision": precision_score,
            "Recall": recall_score
        },
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_features
    )
    return {
        "Demographic Parity Difference": demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_features),
        "Equalized Odds Difference": equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_features),
        #"Predictive Parity Difference": abs(mf.by_group["Precision"].diff().iloc[0])  # difference in PPV
    }

# Compute fairness metrics for both models
fairness_lr = compute_fairness_metrics(y_test, y_pred_lr, A_test)
fairness_rf = compute_fairness_metrics(y_test, y_pred_rf,A_test)

# Combine into a DataFrame
fairness_results = pd.DataFrame([fairness_lr, fairness_rf], index=["Logistic Regression", "Random Forest"])
print(fairness_results)

#output confusion matrix


                     Demographic Parity Difference  Equalized Odds Difference
Logistic Regression                       0.060504                   0.149935
Random Forest                             0.095556                   0.056921


In [ ]:
# Manual fairness metric computations using group-wise analysis

def group_metrics(y_true, y_pred, sensitive_attr, group_val):
    mask = (sensitive_attr == group_val)
    y_true_group = y_true[mask]
    y_pred_group = y_pred[mask]

    tp = ((y_true_group == 1) & (y_pred_group == 1)).sum()
    fp = ((y_true_group == 0) & (y_pred_group == 1)).sum()
    tn = ((y_true_group == 0) & (y_pred_group == 0)).sum()
    fn = ((y_true_group == 1) & (y_pred_group == 0)).sum()

    selection_rate = (y_pred_group == 1).mean()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0 #correct
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0 #correct
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0 #correct

    return selection_rate, precision, recall, fpr

# Wrapper to compute fairness metrics
def compute_fairness(y_true, y_pred, sensitive_attr):
    male_metrics = group_metrics(y_true, y_pred, sensitive_attr, 1)
    female_metrics = group_metrics(y_true, y_pred, sensitive_attr, 0)

    dp_diff = abs(male_metrics[0] - female_metrics[0])  # Demographic Parity: difference in selection rate
    ppv_diff = abs(male_metrics[1] - female_metrics[1])  # Predictive Parity: difference in precision
    eq_odds_diff = abs(male_metrics[2] - female_metrics[2]) + abs(male_metrics[3] - female_metrics[3])  # TPR + FPR diff

    return {
        "Demographic Parity Difference": dp_diff,
        "Predictive Parity Difference": ppv_diff,
        "Equalized Odds Difference (TPR+FPR)": eq_odds_diff
    }

# Compute for both models
fairness_lr_manual = compute_fairness(y_test, y_pred_lr, A_test)
fairness_rf_manual = compute_fairness(y_test, y_pred_rf, A_test)

manual_fairness_df = pd.DataFrame([fairness_lr_manual, fairness_rf_manual],
                                   index=["Logistic Regression", "Random Forest"])
print(manual_fairness_df)

                     Demographic Parity Difference  \
Logistic Regression                       0.060504   
Random Forest                             0.095556   

                     Predictive Parity Difference  \
Logistic Regression                      0.024276   
Random Forest                            0.012949   

                     Equalized Odds Difference (TPR+FPR)  
Logistic Regression                             0.182251  
Random Forest                                   0.112727  


**Goal 3:** Use other classifiers and make a more comprehensive study. Available classifiers: https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html




In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

models = {
    "Logistic Regression": LogisticRegression(max_iter=10000),
    "Random Forest": RandomForestClassifier(n_estimators=30, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Support Vector Machine": SVC(probability=True, random_state=42),
    "Neural Network": MLPClassifier(max_iter=300, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}



In [ ]:
from fairlearn.metrics import MetricFrame, demographic_parity_difference, equalized_odds_difference, selection_rate
from sklearn.metrics import precision_score, recall_score

def compute_fairness_metrics(y_true, y_pred, sensitive_features):
    mf = MetricFrame(
        metrics={
            "Selection Rate": selection_rate,
            "Precision": precision_score,
            "Recall": recall_score
        },
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_features
    )
    #print(mf.by_group["Precision"])
    ppv_values = mf.by_group["Precision"].dropna()
    if len(ppv_values) >= 2:
        ppv_diff = abs(ppv_values.iloc[0] - ppv_values.iloc[1])
    else:
        ppv_diff = np.nan
    #print(ppv_diff)
    return {
        "Demographic Parity Difference": demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_features),
        "predictive rate parity difference": ppv_diff,
        "Equalized Odds Difference": equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_features)
    }

fairness_results = {}
accuracy_results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    fairness = compute_fairness_metrics(y_test, y_pred, A_test)

    fairness_results[name] = fairness
    accuracy_results[name] = acc

# Fairness
fairness_df = pd.DataFrame(fairness_results).T
print("\nFairness Metrics:")
print(fairness_df)

# Accuracy
accuracy_df = pd.DataFrame.from_dict(accuracy_results, orient='index', columns=["Accuracy"])
print("\nAccuracy:")
print(accuracy_df)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:23:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Fairness Metrics:
                        Demographic Parity Difference  \
Logistic Regression                          0.060504   
Random Forest                                0.095556   
Decision Tree                                0.127048   
Support Vector Machine                       0.043185   
Neural Network                               0.079564   
XGBoost                                      0.068846   

                        predictive rate parity difference  \
Logistic Regression                              0.024276   
Random Forest                                    0.012949   
Decision Tree                                    0.058750   
Support Vector Machine                           0.122975   
Neural Network                                   0.020928   
XGBoost                                          0.021037   

                        Equalized Odds Difference  
Logistic Regression                      0.149935  
Random Forest                            0.056921

**Goal 4:** Employ fairness mitigation measures on these classifiers and see if we can improve the metrics. Example- postprocessing: https://fairlearn.org/v0.5.0/api_reference/fairlearn.postprocessing.html


**Goal 5:** More reasonable datsets- repeat the above exersice with datasets that make sense for each notion of fairness (where we can explain to the class that pursuing for example demographic parity is a reasonable goal)

In [ ]:
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.metrics import equalized_odds_difference, MetricFrame, selection_rate
from sklearn.metrics import precision_score


In [ ]:
!pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Use the filename exactly as it appears in the dataset
file_path = "bar_pass_prediction.csv"

# Load the dataset directly into a DataFrame
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "danofer/law-school-admissions-bar-passage",
    file_path
)

print("First 5 records:")
print(df.head())

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from fairlearn.metrics import equalized_odds_difference, MetricFrame, selection_rate
from sklearn.metrics import precision_score

# Drop missing values
df = df.dropna(subset=['bar_passed', 'lsat', 'ugpa', 'sex', 'race'])

# Only keep rows where bar_passed is a valid boolean
df = df[df['bar_passed'].isin([True, False])]

#  STEP 2: Convert bar_passed to numeric (1 = pass, 0 = fail)
df['bar_pass'] = df['bar_passed'].astype(int)

#  STEP 3: Encode categorical features
df['sex'] = LabelEncoder().fit_transform(df['sex'])
df['race'] = LabelEncoder().fit_transform(df['race'])

#  STEP 4: Define features, target, and sensitive attribute
X = df[['lsat', 'ugpa', 'sex', 'race']]
y = df['bar_pass']
A_race = df['race']
A_sex = df['sex']  # Or switch to 'sex' if you want to evaluate by gender
cr = CorrelationRemover(sensitive_feature_ids=['sex', 'race'])
print("Full dataset race counts:")
print(df['race'].value_counts())



# STEP 5: Train-test split
X_train, X_test, y_train, y_test, A_race_train, A_race_test, A_sex_train, A_sex_test = train_test_split(
    X, y, A_race, A_sex, test_size=0.3, random_state=42)

#  STEP 6: Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

cr = CorrelationRemover(sensitive_feature_ids=[2, 3])  # indices of 'sex' and 'race' in X
X_train_cr = cr.fit_transform(X_train_scaled)
X_test_cr = cr.transform(X_test_scaled)

equalized_odds_results = {}
predictive_parity_results = {}


selection_rates_race = {}
selection_rates_sex = {}


for name, model in models.items():
    model.fit(X_train_cr, y_train)
    y_pred = model.predict(X_test_cr)

    eod_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=A_race_test)
    equalized_odds_results[name] = eod_diff

    # Predictive Parity (Precision) Difference by Race
    mf_precision = MetricFrame(
        metrics=precision_score,
        y_true=y_test,
        y_pred=y_pred,
        sensitive_features=A_race_test
    )
    predictive_parity_diff = mf_precision.by_group.max() - mf_precision.by_group.min()
    predictive_parity_results[name] = predictive_parity_diff

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_results[name] = acc

    # Demographic Parity
    dp_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=A_race_test)
    fairness_results[name] = dp_diff

    # Selection rate by race
    metric_race = MetricFrame(
        metrics=selection_rate,
        y_true=y_test,
        y_pred=y_pred,
        sensitive_features=A_race_test
    )
    selection_rates_race[name] = metric_race.by_group

    # Selection rate by sex
    metric_sex = MetricFrame(
        metrics=selection_rate,
        y_true=y_test,
        y_pred=y_pred,
        sensitive_features=A_sex_test
    )
    selection_rates_sex[name] = metric_sex.by_group

print("\n🎯 Accuracy Scores:")
print(pd.Series(accuracy_results))

# Demographic Parity Difference
print("\n⚖️ Demographic Parity Differences:")
print(pd.Series(fairness_results))

print("\n📊 Selection Rates by Race:")
for model_name, rates in selection_rates_race.items():
    print(f"\n{model_name}:")
    print(rates)

print("\n📊 Selection Rates by Sex:")
for model_name, rates in selection_rates_sex.items():
    print(f"\n{model_name}:")
    print(rates)

print("Full dataset race counts:")
print(df['race'].value_counts())

print("\n📊 Equalized Odds Differences by Race:")
print(pd.Series(equalized_odds_results))

# Predictive Parity (Precision) Difference
print("\n📊 Predictive Parity Differences by Race:")
print(pd.Series(predictive_parity_results))



<ipython-input-29-6a296c8dc0a0>:9: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:
   decile1b  decile3  ID  decile1  sex  race  cluster  lsat  ugpa  zfygpa  \
0      10.0     10.0   2     10.0  1.0   7.0      1.0  44.0   3.5    1.33   
1       5.0      4.0   3      5.0  1.0   7.0      2.0  29.0   3.5   -0.11   
2       3.0      2.0  36      3.0  2.0   7.0      3.0  36.0   3.5   -0.64   
3       7.0      4.0  52      7.0  2.0   7.0      3.0  39.0   3.5    0.34   
4       9.0      8.0  55      9.0  2.0   7.0      4.0  48.0   3.5    1.02   

   ...  hisp pass_bar                bar bar_passed  tier   index6040  \
0  ...     0        1  a Passed 1st time       True   4.0  886.842082   
1  ...     0        1  a Passed 1st time       True   2.0  649.999987   
2  ...     0        1  a Passed 1st time       True   3.0  760.526298   
3  ...     0        1  a Passed 1st time       True   3.0  807.894717   
4  ...     0        1  a Passed 1st time       True   5.0  949.999974   

     indxgrp   indxgrp2  dnn_bar_pass_prediction  gpa  
0     g 700+     i 820+  

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:05:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🎯 Accuracy Scores:
Logistic Regression       0.946256
Random Forest             0.939259
Decision Tree             0.927497
Support Vector Machine    0.946554
Neural Network            0.946256
XGBoost                   0.944618
dtype: float64

⚖️ Demographic Parity Differences:
Logistic Regression       0.006623
Random Forest             0.127473
Decision Tree             0.222575
Support Vector Machine    0.002198
Neural Network            0.006593
XGBoost                   0.039560
dtype: float64

📊 Selection Rates by Race:

Logistic Regression:
race
0    1.000000
1    1.000000
2    0.993407
3    1.000000
4    1.000000
5    0.993377
6    0.999642
7    1.000000
Name: selection_rate, dtype: float64

Random Forest:
race
0    0.941176
1    0.973684
2    0.872527
3    0.974576
4    1.000000
5    0.940397
6    0.993909
7    0.948718
Name: selection_rate, dtype: float64

Decision Tree:
race
0    0.764706
1    0.924812
2    0.782418
3    0.915254
4    0.939394
5    0.880795
6    0.987281
7

**Goal 6:** Make a presentation of our findings above (this is part 2 of the ppt)

**Goal 7:** Make a presentation exlaining each notion of group fairness in a consise way (this is part 1 of our ppt)

In [ ]:
print(mf.by_group["Precision"])

NameError: name 'mf' is not defined

In [ ]:
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate, equalized_odds_difference

# Define base classifier
base_estimator = LogisticRegression(solver='liblinear')

# Create the fairness-aware optimizer
mitigator = ExponentiatedGradient(
    estimator=base_estimator,
    constraints=EqualizedOdds(),
    sample_weight_name="sample_weight"
)

# Fit with sensitive features
mitigator.fit(X_train, y_train, sensitive_features=A_train)

# Predict and evaluate
y_pred_mitigated = mitigator.predict(X_test)

# Evaluate accuracy and fairness
acc_mitigated = accuracy_score(y_test, y_pred_mitigated)
eo_diff = equalized_odds_difference(y_test, y_pred_mitigated, sensitive_features=A_test)

print("Accuracy (In-Processing):", acc_mitigated)
print("Equalized Odds Difference:", eo_diff)


Accuracy (In-Processing): 0.924668750930475
Equalized Odds Difference: 0.07692307692307687
